# Topic Modeling and Visualization

Topic modeling using `sklearn` and `pyLDAvis` to visualize topic modeling that works with the results of both`sklearn` and `gensim`.

 <a id="sec1"></a>
 ## 1. Prepare the notebook materials

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Import the needed packages and enable pyLDAvis to work inside the notebook:

In [2]:
import pandas
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

### Load headline and actor description dataset

In [6]:
actor_desc = pandas.read_csv("GPT_actor_blurbs.csv")

In [7]:
# displaying the contents of the CSV file
actor_desc.head(10)

,title,actor_blurb
0,"CNN Soledad OBrien , Liberal Guest Team Up to ...",The primary actor in the headline is Soledad O...
1,A Critical Race Tale Told by Leftist Idiots Si...,The headline does not reference a specific pri...
2,Amazing Benefits of Becoming Salesforce Develo...,The headline does not explicitly reference a p...
3,The race is on ; Candidates address critical i...,"The primary actors in the headline ""The race i..."
4,Journalist declares one - man war against cri...,"The primary actor in the headline is a ""journa..."
5,Reminder : Critical race theory isnt scriptural,The headline does not reference a specific pri...
6,B . C . Court Of Appeal Weighs In On Legal Rep...,"The primary actor in the headline is the ""B.C...."
7,Continued Violence In Kenosha May Tighten 2020...,The headline does not reference a specific pri...
8,Sandia Labs Goes Nuclear On Employee Who Spark...,"The primary actor in the headline is ""Sandia L..."
9,Rebellion Against Critical Race Theory in the ...,"The primary actor in the headline ""Rebellion A..."


In [8]:
blurbs = actor_desc['actor_blurb'].tolist()
print(len(blurbs))

11704


<a id="sec2"></a>
## 2. Convert to document-term matrices

Next, the raw documents are converted into document-term matrix (dtm), initially as raw counts (with `CountVectorizer`) and then as TF-IDF values (with `TfidfVectorizer`).

We're creating two different vectorizers, to notice the difference that different feature representations make in the process of modeling.

In [9]:
# Create the TF vector represetnation, this only counts the terms in each document

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(blurbs)
print(dtm_tf.shape)

(11704, 2188)


Let's look at some of the features (i.e. words in the corpus):

In [10]:
tf_vectorizer.get_feature_names_out()[500:520]

array(['created', 'creating', 'creation', 'creator', 'crenshaw',
       'criminal', 'crisis', 'criteria', 'critic', 'criticism',
       'criticisms', 'criticize', 'criticized', 'criticizes',
       'criticizing', 'critics', 'critique', 'critiques', 'critiquing',
       'crt'], dtype=object)

To be able to compare the results of TF and TFIDF representation, we will use the same parameters.
This is why we will initialize the `TfidfVectorizer` with the parmeters of the `CountVectorizer`.

Initialize and prepare Document-Term-Matrix:

In [13]:
# TF IDF = measures term importance relative to the other documents
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(blurbs)
print(dtm_tfidf.shape)

(11704, 2188)


/Users/annalieb/miniconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:2078: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn(


<a id="sec3"></a>
## 3. Fit Latent Dirichlet Allocation models

We will fit two LDA models, each with 3 topics. 

In [14]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tf.fit(dtm_tf)

lda_tf

LatentDirichletAllocation(n_components=4, random_state=0)

In [15]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=3, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=3, random_state=0)

Given that the models are probability distributions over topics and words, we will focus on their visualization to learn more about them.

<a id="sec4"></a>
## 4. Visualizing the models with pyLDAvis

Note on relevance metric: lambda influences the order in which the words are sorted. Adjust lambda to help with interpretation of the different categories. 

see this link: https://we1s.ucsb.edu/research/we1s-tools-and-software/topic-model-observatory/tmo-guide/tmo-guide-pyldavis/

In [16]:
# the TF representation model

pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.060597 -0.017605       1        1  28.783329
1     -0.167072 -0.213349       2        1  28.133639
0     -0.157162  0.226868       3        1  21.708186
3      0.263637  0.004086       4        1  21.374846, topic_info=             Term         Freq        Total Category  logprob  loglift
1944        state  3543.000000  3543.000000  Default  30.0000  30.0000
651          does  2706.000000  2706.000000  Default  29.0000  29.0000
1919     specific  3095.000000  3095.000000  Default  28.0000  28.0000
1701    reference  2140.000000  2140.000000  Default  27.0000  27.0000
1534    political  3834.000000  3834.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
8        academic   355.405153   735.792276   Topic4  -5.0312   0.8153
1984     suggests   419.151003  1303.880891   Topic4  -4.8662   0.4081
1712    regarding   410.949064  1264.871329   Topic4  -4.8860   0.4187
1212       likely   433.231199  2440.866586   Topic4  -4.8332  -0.1859
676   educational   363.098906  3251.351769   Topic4  -5.0098  -0.6492

[288 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
8         1  0.436264  academic
8         2  0.001359  academic
8         3  0.080186  academic
8         4  0.482473  academic
26        1  0.221438    action
...     ...       ...       ...
2164      3  0.731888      work
2168      1  0.924516   working
2168      2  0.072402   working
2185      1  0.983241  youngkin
2185      2  0.014250  youngkin

[461 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

In [17]:
# The TFIDF representation model
# not as helpful as TF-- could be due to French? Not sure

pyLDAvis.lda_model.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.026436 -0.175294       1        1  37.018356
1     -0.209679  0.069956       2        1  36.719670
0      0.183242  0.105338       3        1  26.261974, topic_info=           Term        Freq       Total Category  logprob  loglift
651        does  524.000000  524.000000  Default  30.0000  30.0000
1701  reference  454.000000  454.000000  Default  29.0000  29.0000
1919   specific  484.000000  484.000000  Default  28.0000  28.0000
1534  political  381.000000  381.000000  Default  27.0000  27.0000
1947     states  212.000000  212.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1529   policies   86.675578  232.034871   Topic3  -4.9471   0.3523
1897     social   67.813409  151.539783   Topic3  -5.1925   0.5330
1530     policy   64.045797  151.682751   Topic3  -5.2497   0.4749
675   education   64.433356  242.139763   Topic3  -5.2436   0.0132
824     florida   59.718501  145.713550   Topic3  -5.3196   0.4450

[214 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
8         1  0.612281  academic
8         2  0.008747  academic
8         3  0.376115  academic
26        1  0.187257    action
26        2  0.765080    action
...     ...       ...       ...
2105      1  0.218357   various
2105      2  0.046791   various
2105      3  0.733054   various
2174      1  0.960602    writer
2185      1  0.987742  youngkin

[323 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])